# Carregando documentos e ampliando o contexto para resposta


In [1]:
# Carregando variáveis de ambiente
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(), override=True)

True

In [2]:
# Inicializando a sessão com o Astra/Cassandra
# A criação da sessão é realizada pelos arquivos cqlsession.py e connect_database.py
from cqlsession import getCQLSession, getCQLKeyspace
cqlMode = 'astra_db' # 'astra_db'/'local'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

In [3]:
# Neste exemplo, vou utilizar os embeddings da OpenAI.
# Aqui, importamos o LLM e os Embeddings
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
llm = OpenAI(temperature=0)
myEmbedding = OpenAIEmbeddings()

In [4]:
# Importando o vector store Cassandra (vamos usar o Astra, mas é o Cassandra)
from langchain.vectorstores.cassandra import Cassandra

In [5]:
# Importando o índice, separadores de texto, e auxiliares para processamendo dos documentos
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader, PyPDFLoader

In [42]:
# Criando o índice
table_name = 'vs_astra_openai'
keyspace = 'demo'

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Cassandra,
    embedding=myEmbedding,
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=30,
    ),
    vectorstore_kwargs={
        'session': session,
        'keyspace': keyspace,
        'table_name': table_name,
    },
)

In [43]:
# Vamos usar o texto o conto "O Alienista", de Machado de Assis.
# PDF disponível em http://www.dominiopublico.gov.br/download/texto/bv000231.pdf
loader = TextLoader('./o_alienista.txt', encoding='utf8')

In [44]:
# Carregando o documento para o Astra
index = index_creator.from_loaders([loader])

In [45]:
cqlSelect = f'SELECT * FROM {keyspace}.{table_name} LIMIT 3;'  # (Not a production-optimized query ...)
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    document_id:      {row.document_id}')
    print(f'    embedding_vector: {str(row.embedding_vector)[:64]} ...')
    print(f'    document:         {row.document[:64]} ...')
    print(f'    metadata_blob:    {row.metadata_blob}')

print('\n...')


Row 0:
    document_id:      722a59223c904be29a340eed04b961ed
    embedding_vector: [0.0021424368023872375, -0.002469730330631137, 0.016110857948660 ...
    document:         O alienista atendeu então; os gritos aproximavam-se, terríveis,  ...
    metadata_blob:    {"source": "./o_alienista.txt"}

Row 1:
    document_id:      f2b10762b0d944e6a2f21505afe60394
    embedding_vector: [0.016153687611222267, 0.009189954958856106, 0.00985510647296905 ...
    document:         porque Napoleão não provou a Casa Verde, o barbeiro achou prefer ...
    metadata_blob:    {"source": "./o_alienista.txt"}

Row 2:
    document_id:      63564952c9514d92af4284794a881b3f
    embedding_vector: [-0.0026760578621178865, 0.022114384919404984, 0.018191084265708 ...
    document:         inexplorada. Simão Bacamarte compreendeu que a ciência lusitana, ...
    metadata_blob:    {"source": "./o_alienista.txt"}

...


In [53]:
query = "Quem foi Simão Bacamarti?"
index.query_with_sources(query, llm=llm)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


{'question': 'Quem foi Simão Bacamarti?',
 'answer': ' Simão Bacamarte foi um médico brasileiro, filho da nobreza da terra, que estudou em Coimbra e Pádua. Ele foi considerado o maior dos médicos do Brasil, de Portugal e das Espanhas.\nSource: ./o_alienista.txt',
 'sources': ''}